In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\programdata\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold


from matplotlib import style
style.use("fivethirtyeight")

from matplotlib import rcParams
rcParams['figure.figsize'] = 10, 8

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# nx.from_pandas_edgelist
# Set features. This will also be used as your x values.
# test_features = df[['koi_period', 'koi_time0bk', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]
# X = test_features

X = df.drop('koi_disposition', axis=1)
X = X.head(100)
# X = X.values.tolist()
# X

# X = df.drop('koi_disposition', axis=1)
# X = df.head(200)
# X.head()

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:


# Assign koi_disposition to y
Y = df[['koi_disposition']]


# # Assign koi_disposition to y and Remove spaces btw words
# Y = df['koi_disposition'].apply(lambda str : str.replace(" ", ""))

# 200 samples for test
Y = Y.head(100)

# LABEL ENCODE Y
# Import required module
from sklearn.preprocessing import LabelEncoder

# LabelEncoder
le = LabelEncoder()

# Create an object of the label encoder class
labelencoder = LabelEncoder()

# apply "le.fit_transform"
y = Y.apply(le.fit_transform)

# Change the shape of Y
# y=np.array(y['koi_disposition'])
y

# Apply labelencoder object on columns
# labelencoder.fit_transform(data.ix[:, 1:])   # First column does not need to be encoded

,koi_disposition
0,1
1,2
2,2
3,1
4,1
...,...
95,1
96,1
97,1
98,1


In [7]:
# Split data into train and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [8]:
# X_train

In [9]:
y_train

,koi_disposition
68,1
75,2
21,1
94,1
8,1
...,...
6,1
26,1
60,1
50,0


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

# Create variables to hold the scaled train & test data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

# Train the Model



In [11]:
model1 = SVC(kernel="linear")
model1


# print(f"Training Data Score: {model1.score(X_train_scaled, y_train)}")
# print(f"Testing Data Score: {model1.score(X_test_scaled, y_test)}")

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [12]:
# Create the GridSearchCV model
param_grid = {'C': [0.01, 1, 5],
              'gamma': [0.0005, 0.001, 0.005]}

cv = KFold(n_splits=2, shuffle=False, random_state=None)
grid = GridSearchCV(model1, param_grid, verbose=3, scoring='accuracy', cv=cv)


# # Create the GridSearchCV model
# param_grid = {'C': [0.01, 1, 5],
#               'gamma': [0.0005, 0.001, 0.005],
#              'n_splits_': [2]}
# kf = KFold(n_splits=2, shuffle=False, random_state=None)
# newxtrain=kf.split(np.array(X_train['koi_period']))
# #grid = GridSearchCV(model1, param_grid, verbose=3, scoring='f1', cv=cv)

In [17]:
# Train the model with GridSearch
grid.fit(X_train, y_train)
# grid.fit(X_train_scaled, y_train_scaled)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] C=0.01, gamma=0.0005 ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Unknown label type: 'continuous'

In [14]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [15]:
# List best parameters and best score
print(grid.best_params_)
print(grid.best_score_)

{'C': 0.01, 'gamma': 0.0005}
0.72


# Save the Model

In [18]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'testModel1.sav'
joblib.dump(GridSearchCV, filename)

['testModel1.sav']